In [8]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd


In [9]:
codes = pd.read_csv('London postcode districts.xlsx - PC DIST.csv')
codes.tail(1)

,Postcode district,Local Areas,Borough
267,WC2R,Somerset House,Westminster


In [10]:
def rightMove(i, Trans_type, website):
    s = Service("C:\\Users\\user\\Downloads\\Set ups\\chromedriver_win32\\chromedriver.exe")
    #s = Service("C:\\Users\\akint\\Downloads\\Set up files\\chromedriver.exe")
    driver = webdriver.Chrome(service= s)

    driver.get('https://www.rightmove.co.uk/')
    driver.maximize_window()
    time.sleep(2)

    time.sleep(1)
    search = driver.find_element(By.XPATH, '/html/body/div[3]/div[2]/main/div[1]/div/div/div/div/input')
    search.send_keys(postcode)

    time.sleep(2)
    driver.find_element(By.XPATH, '/html/body/div[3]/div[2]/main/div[1]/div/div/div/button[1]').click()

    time.sleep(2)
    driver.find_element(By.XPATH, '/html/body/div[3]/div[2]/div/div[1]/div/form/fieldset[2]/div[4]/button').click()

    Trans_type = []
    address = []
    types = []
    bedrooms = []
    bathrooms = []
    prices = []
    desc = []
    date_added = []
    agent_list = []
    property_url = []
    website = []
    
    df = pd.DataFrame()
    
    i = 1
    while True:
        time.sleep(2)
        print("{} {} {} {}".format('scraping page', i,'from', postcode ))
        address_list = driver.find_elements(by='class name', value='propertyCard-address')
        type_list = driver.find_elements(By.XPATH, "//div/div/div[4]/div[1]/div[2]/a/div/span[1]")
        bedroom_list = driver.find_elements(By.XPATH, '//div/div/div[4]/div[1]/div[2]/a/div/span[3]')
        bathroom_list = driver.find_elements(By.XPATH, '//div/div/div[4]/div[1]/div[2]/a/div/span[5]')
        price_list = driver.find_elements(By.XPATH, '//div/div/div[3]/div/a/div[1]')
        desc_list = driver.find_elements(By.XPATH, '//div/div/div[4]/div[1]/div[3]/a/span/span')
        date_added_list = driver.find_elements(By.XPATH, '//div/div/div[4]/div[2]/div/span')
        agent_list_list = driver.find_elements(By.XPATH, '//div/div/div[4]/div[2]/div')
        property_url_list = driver.current_url
        Trans_type_list = Trans_type
        website_list = website
        for address_item, type_item, bedroom_item, bathroom_item, price_item, desc_item, date_added_item, agent_list_item in zip(address_list, type_list, bedroom_list, bathroom_list, price_list, desc_list, date_added_list, agent_list_list):
            address.append(address_item.text)
            types.append(type_item.text)
            bedrooms.append(bedroom_item.text)
            bathrooms.append(bathroom_item.text)
            prices.append(price_item.text)
            desc.append(desc_item.text)
            date_added.append(date_added_item.text)
            agent_list.append(agent_list_item.text)
            property_url.append(property_url_list)
            Trans_type.append(Trans_type_list)
            website.append(website_list)
        next_botton = driver.find_element(By.XPATH, '/html/body/div[3]/div[2]/div[1]/div[3]/div/div/div/div[3]/button/span')
        last_botton_check = driver.find_element(By.XPATH, '/html/body/div[3]/div[2]/div[1]/div[3]/div/div/div/div[3]/button')
        html = last_botton_check.get_attribute('outerHTML') 
        if 'disabled' in html:
            print('------------------------------- SCRAPING COMPLETED FOR ' + postcode)
            break
        next_botton.click()
        i += 1
        time.sleep(2)
        
    df1 = pd.DataFrame({'Location': postcode, 'Tansaction_Type': 'Sales','Property_Type':types, 'Address' :address, 'Bedrooms': bedrooms, 'Bathrooms':bathrooms, 'Price':prices, 'Description': desc, 'Listing_Date':date_added, 'Agent':agent_list, 'Listing_Source': 'RightMove', 'listing_URL':property_url})
    df1.insert(0, 'Unique_Id', [f'{postcode}{i+1:05d}' for i in range(len(df1))])
    print(df1.shape)
    
    df2 = pd.concat([df, df1], ignore_index=True)
    return df2



In [11]:
for postcode in codes.tail(2)['Postcode district']:
    data = rightMove('postcode', 'Sales', 'RightMove')
print('-------------------------------    ------------------------------- WEB SCRAPING COMPLETED')

scraping page 1 from WC2N
------------------------------- SCRAPING COMPLETED FOR WC2N
(18, 13)
scraping page 1 from WC2R
scraping page 2 from WC2R
scraping page 3 from WC2R
scraping page 4 from WC2R
scraping page 5 from WC2R
scraping page 6 from WC2R
------------------------------- SCRAPING COMPLETED FOR WC2R
(139, 13)
-------------------------------    ------------------------------- WEB SCRAPING COMPLETED


In [12]:
data.shape

(139, 13)

In [13]:
data

,Unique_Id,Location,Tansaction_Type,Property_Type,Address,Bedrooms,Bathrooms,Price,Description,Listing_Date,Agent,Listing_Source,listing_URL
0,WC2R00001,WC2R,Sales,Apartment,"Strand, Covent Garden, WC2R",1,1,"£1,500,000","A stunning one bedroom, one bathroom apartment...",Added on 17/11/2020,Allow all cookies,RightMove,https://www.rightmove.co.uk/property-for-sale/...
1,WC2R00002,WC2R,Sales,Apartment,"190, The Strand, WC2R",3,3,"£6,500,000",A spacious and bright three bedroom apartment ...,Added on 16/03/2023,"Added on 17/11/2020 by CBRE Residential, West End",RightMove,https://www.rightmove.co.uk/property-for-sale/...
2,WC2R00003,WC2R,Sales,Apartment,"Strand, London",3,3,"£6,000,000",3bedrooms 3bathrooms.,Added on 14/03/2023,"Added on 16/03/2023 by Citian & Partners, London",RightMove,https://www.rightmove.co.uk/property-for-sale/...
3,WC2R00004,WC2R,Sales,Apartment,"Strand, London, WC2R",3,3,"£5,999,995",A spacious and bright three bedroom apartment ...,Added on 22/07/2022,Added on 14/03/2023 by CHARLES WILLIAM PROPERT...,RightMove,https://www.rightmove.co.uk/property-for-sale/...
4,WC2R00005,WC2R,Sales,Flat,"The Strand, The Strand, London, WC2R",3,3,"£5,500,000",This luxurious 3 bedroom apartment is set in a...,Reduced yesterday,"Added on 22/07/2022 by Life Residential, Count...",RightMove,https://www.rightmove.co.uk/property-for-sale/...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
134,WC2R00135,WC2R,Sales,Flat,"The Strand, Covent Garden, WC2R",4,2,"£1,150,000",Great opportunity to develop this spacious thi...,Added on 13/12/2022,"Added on 13/12/2022 by Chase Evans, City & Ald...",RightMove,https://www.rightmove.co.uk/property-for-sale/...
135,WC2R00136,WC2R,Sales,Flat,"Clement House, 190 Strand, Covent Garden, Lond...",1,1,"£1,075,000",A rare opportunity to purchase a studio flat i...,ONLINE VIEWING,ONLINE VIEWING,RightMove,https://www.rightmove.co.uk/property-for-sale/...
136,WC2R00137,WC2R,Sales,Studio,"The Strand, The Strand, London, WC2R",1,1,"£1,075,000",Found within a prestigious new development on ...,Added on 19/11/2020,"Added on 19/11/2020 by Knight Frank, Mayfair",RightMove,https://www.rightmove.co.uk/property-for-sale/...
137,WC2R00138,WC2R,Sales,Flat,"Charles Apartments, 1 Bull Inn Court, Covent G...",1,1,"£1,075,000",A rare opportunity to purchase a one bedroom f...,Added on 15/03/2023,"Added on 15/03/2023 by Benham & Reeves, Canary...",RightMove,https://www.rightmove.co.uk/property-for-sale/...


In [35]:
# Create a pandas dataframe from the list of records
df1 = pd.DataFrame({'Location': postcode, 'Tansaction_Type': 'Sales','Property_Type':types, 'Address' :address, 'Bedrooms': bedrooms, 'Bathrooms':bathrooms, 'Price':prices, 'Description': desc, 'Listing_Date':date_added, 'Agent':agent_list, 'Listing_Source': 'RightMove', 'listing_URL':property_url})
df1.insert(0, 'Unique_Id', [f'BR1{i+1:05d}' for i in range(len(df1))])

In [37]:
df1

,Unique_Id,Location,Tansaction_Type,Property_Type,Address,Bedrooms,Bathrooms,Price,Description,Listing_Date,Agent,Listing_Source,listing_URL


In [198]:
time.sleep(2)

Trans_type = []
address = []
types = []
bedrooms = []
bathrooms = []
prices = []
desc = []
date_added = []
agent_list = []
property_url = []
website = []

i = 1
while True:
    time.sleep(2)
    print('scraping page', i)
    address_list = driver.find_elements(by='class name', value='propertyCard-address')
    type_list = driver.find_elements(By.XPATH, "//div/div/div[4]/div[1]/div[2]/a/div/span[1]")
    bedroom_list = driver.find_elements(By.XPATH, '//div/div/div[4]/div[1]/div[2]/a/div/span[3]')
    bathroom_list = driver.find_elements(By.XPATH, '//div/div/div[4]/div[1]/div[2]/a/div/span[5]')
    price_list = driver.find_elements(By.XPATH, '//div/div/div[3]/div/a/div[1]')
    desc_list = driver.find_elements(By.XPATH, '//div/div/div[4]/div[1]/div[3]/a/span/span')
    date_added_list = driver.find_elements(By.XPATH, '//div/div/div[4]/div[2]/div/span')
    agent_list_list = driver.find_elements(By.XPATH, '//div/div/div[4]/div[2]/div')
    property_url_list = driver.current_url
    Trans_type_list = 'Sales'
    website_list = 'Rightmove'
    for address_item, type_item, bedroom_item, bathroom_item, price_item, desc_item, date_added_item, agent_list_item in zip(address_list, type_list, bedroom_list, bathroom_list, price_list, desc_list, date_added_list, agent_list_list):
        address.append(address_item.text)
        types.append(type_item.text)
        bedrooms.append(bedroom_item.text)
        bathrooms.append(bathroom_item.text)
        prices.append(price_item.text)
        desc.append(desc_item.text)
        date_added.append(date_added_item.text)
        agent_list.append(agent_list_item.text)
        property_url.append(property_url_list)
        Trans_type.append(Trans_type_list)
        website.append(website_list)
    next_botton = driver.find_element(By.XPATH, '/html/body/div[3]/div[2]/div[1]/div[3]/div/div/div/div[3]/button/span')
    last_botton_check = driver.find_element(By.XPATH, '/html/body/div[3]/div[2]/div[1]/div[3]/div/div/div/div[3]/button')
    html = last_botton_check.get_attribute('outerHTML') 
    if 'disabled' in html:
        print('Scraping complete for this Postcode')
        break
    next_botton.click()
    i += 1
    time.sleep(2)


scraping page 1
scraping page 2
scraping page 3
scraping page 4
scraping page 5
scraping page 6
scraping page 7
scraping page 8
scraping page 9
scraping page 10
scraping page 11
Scraping complete for this Postcode


In [5]:
# Create a pandas dataframe from the list of records
df = pd.DataFrame({'Location': postcode, 'Tansaction_Type': Trans_type,'Property_Type':types, 'Address' :address, 'Bedrooms': bedrooms, 'Bathrooms':bathrooms, 'Price':prices, 'Description': desc, 'Listing_Date':date_added, 'Agent':agent_list, 'Listing_Source': website, 'listing_URL':property_url})
df.insert(0, 'Unique_Id', [f'BR1{i+1:05d}' for i in range(len(df))])

NameError: name 'Trans_type' is not defined

In [269]:
df.shape

(251, 13)

In [270]:
df

,Unique_Id,Location,Tansaction_Type,Property_Type,Address,Bedrooms,Bathrooms,Price,Description,Listing_Date,Agent,Listing_Source,listing_URL
0,BR100001,BR1,Sales,Apartment,"Mill Pond Road, Dartford",2,1,"£325,000",Looking for a beautiful new build apartment wi...,Reduced on 05/12/2022,Allow all cookies,Rightmove,https://www.rightmove.co.uk/property-for-sale/...
1,BR100002,BR1,Sales,Detached,"Shears Close, Wilmington",6,3,"£1,250,000",NEW INSTRUCTION Stunning 5/6 bed large detache...,Added on 18/04/2023,"Reduced on 05/12/2022 by haart, Dartford",Rightmove,https://www.rightmove.co.uk/property-for-sale/...
2,BR100003,BR1,Sales,Detached,"Princes Road, Dartford",4,2,"£740,000",A substantial double fronted four-bedroom deta...,Reduced on 18/04/2023,"Added on 18/04/2023 by Harpers & Co, Bexley",Rightmove,https://www.rightmove.co.uk/property-for-sale/...
3,BR100004,BR1,Sales,Detached,"Melrose Avenue, Braeburn Park, Crayford, Kent",4,4,"£725,000","* GUIDE PRICE £725,000 TO £750,000 * FOUR BEDR...",Added on 26/12/2022,"Reduced on 18/04/2023 by Chattertons, New Eltham",Rightmove,https://www.rightmove.co.uk/property-for-sale/...
4,BR100005,BR1,Sales,Detached,"Tower Road, Dartford, Kent, DA1",5,1,"£650,000",Located within a five minute walk of both Dart...,Added on 28/02/2023,Added on 26/12/2022 by Livermores The Estate A...,Rightmove,https://www.rightmove.co.uk/property-for-sale/...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,BR100247,BR1,Sales,Maisonette,"Mitchell Close, Wilmington, DA1",2,1,"£150,000",AUCTION NOW ENDED - PROPERTY SOLD ! FOR SALE B...,Reduced on 09/02/2023,"Added on 21/04/2023 by Homewise, Worthing",Rightmove,https://www.rightmove.co.uk/property-for-sale/...
247,BR100248,BR1,Sales,Flat,"Knights Manor Way, Dartford, Kent, DA1",1,1,"£147,500",Robinson Jackson are pleased to offer this gro...,Added on 28/02/2023,"Reduced on 09/02/2023 by eXp UK, South East",Rightmove,https://www.rightmove.co.uk/property-for-sale/...
248,BR100249,BR1,Sales,Flat,"Churchill Close, Dartford, Kent",1,1,"£145,000",Buyers Fees apply - see brochure for details. ...,ONLINE VIEWING,"Added on 28/02/2023 by Robinson Jackson, Dartford",Rightmove,https://www.rightmove.co.uk/property-for-sale/...
249,BR100250,BR1,Sales,Apartment,"Fulwich Road, Dartford",1,1,"£140,000","GUIDE PRICE £140,000 to £160,000 * TOP FLOOR C...",Reduced on 25/04/2023,ONLINE VIEWING,Rightmove,https://www.rightmove.co.uk/property-for-sale/...
